In [1]:
import os
import math
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
cwd = os.getcwd()
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from matplotlib.lines import Line2D
pd.options.display.max_columns = None

In [3]:
plt.style.use(['science'])
os.environ["PATH"] += os.pathsep + '/Library/TeX/texbin'
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [4]:
os.chdir(os.path.join(cwd,'../../..'))
import ExMAS.main
import ExMAS.utils
from ExMAS.utils import inData as inData
from ExMAS.main import matching
import json

In [5]:
EXPERIMENT_NAME = 'res400_4'
KPIs = pd.read_csv('ExMAS/extras/game_results/{}_KPIs.csv'.format(EXPERIMENT_NAME), index_col = 0)
rides = pd.read_csv('ExMAS/extras/game_results/{}_rides.csv'.format(EXPERIMENT_NAME), index_col = 0)
rm = pd.read_csv('ExMAS/extras/game_results/{}_rm.csv'.format(EXPERIMENT_NAME), index_col = [0,1])
del rides['row']

FileNotFoundError: [Errno 2] No such file or directory: 'ExMAS/extras/game_results/res400_4_KPIs.csv'

In [6]:
PRICINGS = ['UNIFORM','EXTERNALITY','RESIDUAL','SUBGROUP']

for i,PRICING in enumerate(PRICINGS):
    
    for j, ALGO in enumerate(['EXMAS', 'TNE', 'HERMETIC', 'RUE', 'RSIE', 'TSE']):
        col_name = "pruned_Pricing-{}_Pruning-['{}']".format(PRICING, ALGO)
        rm[col_name] = rm.apply(lambda r: rides.loc[r.ride][col_name], axis = 1)

NameError: name 'rm' is not defined

-----
groups that are part of ExMAS solution under 'SUBGROUP' pricing and are later pruned by the Hermetic rule

In [16]:
ALGO = 'EXMAS'
PRICING = 'SUBGROUP'
minmax = 'min'
matching_column_EXMAS = "Experiment-{}_Pricing-{}_Objective-['total_group_cost']_Pruning-['{}']_minmax-{}".format(EXPERIMENT_NAME,PRICING,ALGO,minmax)
pruning_column_EXMAS = "pruned_Pricing-{}_Pruning-['{}']".format(EXPERIMENT_NAME,PRICING,ALGO)

ALGO = 'HERMETIC'
PRICING = 'SUBGROUP'
price_column = PRICING
minmax = 'min'
matching_column_HERMETIC = "Experiment-{}_Pricing-{}_Objective-['total_group_cost']_Pruning-['{}']_minmax-{}".format(EXPERIMENT_NAME,PRICING,ALGO,minmax)
pruning_column_HERMETIC = "pruned_Pricing-{}_Pruning-['{}']".format(PRICING,ALGO)
wrongs = rides[(rides[matching_column_EXMAS]==True) & (rides[pruning_column_HERMETIC]==False)]

In [66]:
wrongs[['SUBGROUP','indexes', pruning_column_HERMETIC, matching_column_EXMAS, matching_column_HERMETIC]]

,SUBGROUP,indexes,pruned_Pricing-SUBGROUP_Pruning-['HERMETIC'],Experiment-res400_4_Pricing-SUBGROUP_Objective-['total_group_cost']_Pruning-['EXMAS']_minmax-min,Experiment-res400_4_Pricing-SUBGROUP_Objective-['total_group_cost']_Pruning-['HERMETIC']_minmax-min
614,8.994700,"[36, 147]",False,1,0
762,7.058850,"[50, 56]",False,1,0
978,11.110450,"[176, 215]",False,1,0
1118,10.523150,"[28, 380]",False,1,0
1234,12.830700,"[172, 239]",False,1,0
1421,6.104750,"[159, 248]",False,1,0
1457,15.446950,"[157, 399]",False,1,0
1795,12.275075,"[256, 290, 397]",False,1,0
1830,20.675600,"[121, 13, 282]",False,1,0
1905,15.065975,"[30, 18, 169]",False,1,0


let's take one: `1830`

In [60]:
rides.iloc[:, : 17].loc[1830:1830]

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,lambda_r,PassHourTrav_ns,selected,treqs,deps,delays,ttravs
1830,"[121, 13, 282]",34.5437,1077,31,"[12.032, 13.479400000000002, 9.032300000000001]","[53.0, 94, 207, 594, 75, 107]","[121, 13, 282]","[282, 121, 13]",3,1830,0.522395,2255,1,[166 19 390],"[53.0, 147.0, 354.0]",[-113. 128. -36.],"[970, 983, 776]"


In [63]:
rides.loc[1830:1830][['costs_veh','costs_user','costs_total']]

,costs_veh,costs_user,costs_total
1830,9.66985,11.00575,20.6756


In [61]:
rm[rm.ride==1830]

ride  traveller  shared  degree  treq  ride_time  dist  ttrav  \
1830 121  1830        121    True       3   166       1077  6326    790   
     13   1830         13    True       3    19       1077  6971    871   
     282  1830        282    True       3   390       1077  4759    594   

          ttrav_sh  delay  distance  cost_veh  cost_user  total_group_cost  \
1830 121       970 -113.0      8616   9.66985    3.98825           20.6756   
     13        983  128.0      8616   9.66985    4.11250           20.6756   
     282       776  -36.0      8616   9.66985    2.90500           20.6756   

          cost_efficiency  cost_single  total_singles  residual_user  \
1830 121         6.891867     10.12450       29.04525       -8.36965   
     13          6.891867     11.06005       29.04525       -8.36965   
     282         6.891867      7.86070       29.04525       -8.36965   

           UNIFORM  desired_UNIFORM  EXTERNALITY  desired_EXTERNALITY  \
1830 121  6.891867         6.317925      4.78975              0.00000   
     13   6.891867         6.891867      7.44100              5.86005   
     282  6.891867         6.246650      4.69105              3.11010   

          RESIDUAL  desired_RESIDUAL  price_subgroup  SUBGROUP  \
1830 121  7.207034          6.773853        9.699225  8.144851   
     13   7.872997          7.872997        8.297125  6.599120   
     282  5.595569          5.259245        7.138450  5.931628   

          desired_SUBGROUP  \
1830 121          6.773853   
     13           7.872997   
     282          5.259245   

          Experiment-res400_4_Pricing-UNIFORM_Objective-['total_group_cost']_Pruning-['EXMAS']_minmax-min  \
1830 121                                                  1                                                 
     13                                                   1                                                 
     282                                                  1                                                 

          bestpossible_UNIFORM  \
1830 121              6.317925   
     13               6.891867   
     282              6.246650   

          Experiment-res400_4_Pricing-UNIFORM_Objective-['total_group_cost']_Pruning-['EXMAS']_minmax-max  \
1830 121                                                  0                                                 
     13                                                   0                                                 
     282                                                  0                                                 

          Experiment-res400_4_Pricing-UNIFORM_Objective-['total_group_cost']_Pruning-['TNE']_minmax-min  \
1830 121                                                  1                                               
     13                                                   1                                               
     282                                                  1                                               

          Experiment-res400_4_Pricing-UNIFORM_Objective-['total_group_cost']_Pruning-['TNE']_minmax-max  \
1830 121                                                  0                                               
     13                                                   0                                               
     282                                                  0                                               

          Experiment-res400_4_Pricing-UNIFORM_Objective-['total_group_cost']_Pruning-['HERMETIC']_minmax-min  \
1830 121                                                  0                                                    
     13                                                   0                                                    
     282                                                  0                                                    

          Experiment-res400_4_Pricing-UNIFORM_Objective-['total_group_cost']_Pruning-['HERMETIC']_minmax-max  

In [69]:
rm[rm.ride==1830][['ttrav_sh','delay','cost_user','SUBGROUP']]

ttrav_sh  delay  cost_user  SUBGROUP
1830 121       970 -113.0    3.98825  8.144851
     13        983  128.0    4.11250  6.599120
     282       776  -36.0    2.90500  5.931628

is it hermetic? 

**No**


travellers 121 and 282 prefer to ride inm group 1384

In [71]:
def hermetic(ride):
    # checks if ride is hermetic
    G = ride.name
    ride.subgroups = [13,  121,  282,  667,  682, 1384, 1392, 1405, 1830]  # just to parse old data
    
    prices_G = rm.loc[G, :][[price_column]]  # prices of travellers in G
    hermetic = True # assume it is hermetic
    for H in ride.subgroups:  # iterate all subgroups
        # determine whether there exists at least one traveller who wants to use G rather than H
        df = rm.loc[H, :][[price_column]].join(prices_G, lsuffix='_H', rsuffix='_G') # prices in subgroup
        df['surplus'] = df.iloc[:, 0] - df.iloc[:, 1]  # difference in prices
        if df.surplus.max() < 0: # if all are happy in subgroup
            hermetic = False # raise exit flag            
            return hermetic # return
    return hermetic
ride = rides.loc[1830]
hermetic(ride)

False

In [74]:
rides.loc[1830].subgroups

'[  13  121  282  667  682 1384 1392 1405 1830]'

In [73]:
G = 1830
rm.loc[G, :][['ttrav_sh','delay','cost_user','SUBGROUP']]

,ttrav_sh,delay,cost_user,SUBGROUP
121,970,-113.0,3.98825,8.144851
13,983,128.0,4.11250,6.599120
282,776,-36.0,2.90500,5.931628


In [72]:
H = 1384
rm.loc[H, :][['ttrav_sh','delay','cost_user','SUBGROUP']]

,ttrav_sh,delay,cost_user,SUBGROUP
121,807,50.5,3.089625,7.670928
282,669,-35.5,2.527875,5.563672


---
(c) Rafał Kucharski, Delft, 2021